# Vehicle Detection and Counting using YOLOv8

This notebook implements a vehicle detection and counting system using YOLOv8. The model will:
1. Detect vehicles in traffic images
2. Classify them as either cars ('mobil') or motorcycles ('motor')
3. Count total vehicles and per-category counts
4. Visualize the results with bounding boxes and counts

# 1. Setup and Dependencies

First, let's install and import all required libraries

In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from pathlib import Path

# Check if CUDA is available
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

# 2. Load and Preprocess Dataset

Let's load our dataset configuration and examine its structure

In [ ]:
# Set paths
DATASET_PATH = Path("vehicle-detection.v21i.yolov11")
DATA_YAML = DATASET_PATH / "data.yaml"

# Load and display dataset configuration
import yaml
with open(DATA_YAML, 'r') as f:
    data_config = yaml.safe_load(f)
    
print("Dataset Configuration:")
print(f"Number of classes: {data_config['nc']}")
print(f"Class names: {data_config['names']}")
print("\nData splits:")
print(f"Training: {data_config['train']}")
print(f"Validation: {data_config['val']}")
print(f"Testing: {data_config['test']}")

# 3. Configure YOLO Model

Now we'll set up the YOLOv8 model with appropriate configuration for our vehicle detection task

In [ ]:
# Initialize YOLO model
model = YOLO('yolov8n.pt')  # Load the smallest YOLOv8 model for faster training

# Configure model parameters
model.args.update({
    'data': str(DATA_YAML),  # dataset config file
    'epochs': 50,  # number of epochs
    'imgsz': 640,  # image size
    'batch': 16,  # batch size
    'device': device,  # device to use (cuda/cpu)
    'workers': 4,  # number of worker threads
    'patience': 10,  # early stopping patience
})

# 4. Train YOLO Model

Let's train our model on the vehicle dataset

In [ ]:
# Train the model
results = model.train()

# Display training results
print("\nTraining Results:")
print(f"Best mAP50: {results.maps[50]:.3f}")
print(f"Best mAP50-95: {results.maps[0]:.3f}")

# Save the trained model
model.export(format='onnx')

# 5. Vehicle Detection and Counting Functions

Let's implement functions to process the model outputs and count vehicles by type

In [ ]:
def process_detections(results, conf_threshold=0.25):
    """
    Process YOLO detection results and count vehicles by type
    """
    counts = {
        'mobil': 0,  # cars
        'motor': 0,  # motorcycles
        'total': 0
    }
    
    # Process each detection
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Get confidence score
            conf = float(box.conf)
            if conf < conf_threshold:
                continue
                
            # Get class id and increment counter
            cls = int(box.cls)
            class_name = data_config['names'][cls]
            counts[class_name] += 1
            counts['total'] += 1
            
    return counts

def draw_detections(image, results, counts, conf_threshold=0.25):
    """
    Draw bounding boxes and labels on the image
    """
    # Make a copy of the image
    img = image.copy()
    
    # Colors for different classes (BGR format)
    colors = {
        'mobil': (0, 255, 0),  # Green for cars
        'motor': (255, 0, 0)   # Blue for motorcycles
    }
    
    # Draw detections
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Get confidence score
            conf = float(box.conf)
            if conf < conf_threshold:
                continue
                
            # Get box coordinates and class
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls)
            class_name = data_config['names'][cls]
            
            # Draw bounding box
            color = colors[class_name]
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            
            # Add label
            label = f"{class_name} {conf:.2f}"
            cv2.putText(img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    # Add count summary
    summary = f"Total: {counts['total']} | Cars: {counts['mobil']} | Motorcycles: {counts['motor']}"
    cv2.putText(img, summary, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    return img

# 6. Model Inference and Visualization

Let's test our model on some images from the test set

In [ ]:
def process_image(image_path, model, conf_threshold=0.25):
    """
    Process a single image and return detection results with visualization
    """
    # Read image
    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Could not read image: {image_path}")
    
    # Run inference
    results = model.predict(img, conf=conf_threshold)
    
    # Process detections
    counts = process_detections(results, conf_threshold)
    
    # Draw results
    output_img = draw_detections(img, results, counts, conf_threshold)
    
    return output_img, counts

# Test on a sample image
test_image_path = list(Path(data_config['test']).glob('*.jpg'))[0]
output_img, counts = process_image(test_image_path, model)

# Display results
plt.figure(figsize=(15, 10))
plt.imshow(cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Vehicle Detection Results')
plt.show()

print("\nDetection Results:")
print(f"Total vehicles: {counts['total']}")
print(f"Cars: {counts['mobil']}")
print(f"Motorcycles: {counts['motor']}")

# 7. Testing on New Images

Here's how you can use the model to detect and count vehicles in your own traffic images:

In [ ]:
# Function to process a new image
def detect_vehicles_in_image(image_path):
    """
    Detect and count vehicles in a new image
    """
    try:
        output_img, counts = process_image(image_path, model)
        
        # Display results
        plt.figure(figsize=(15, 10))
        plt.imshow(cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title('Vehicle Detection Results')
        plt.show()
        
        print("\nDetection Results:")
        print(f"Total vehicles: {counts['total']}")
        print(f"Cars: {counts['mobil']}")
        print(f"Motorcycles: {counts['motor']}")
        
    except Exception as e:
        print(f"Error processing image: {str(e)}")

# Example usage:
# detect_vehicles_in_image('path_to_your_image.jpg')